# Explore biosamples

In [1]:
import sys
import asyncio
import aiohttp
import logging
from pprint import pprint

sys.path.insert(0, "../scripts")
logger = logging.getLogger(__name__)

# custom supercronic modules
import helpers
from helpers.backend import CONNECTOR, get_ruleset, CDPConverter, put_record, BACKEND_URL
from helpers.biosamples import get_biosample_record

Custom settings valid for ipython notebook (which is executed outside docker contaners):

In [2]:
BACKEND_URL = "http://localhost:26081/backend"
helpers.backend.BACKEND_URL = BACKEND_URL

In [3]:
async def get_converter(session):
    # Get rules from github. Start a task
    logger.info("Getting ruleset")
    ruleset_task = asyncio.create_task(get_ruleset(session))
    standard_rules, organism_rules, specimen_rules = await ruleset_task
    converter = CDPConverter(
            standard_rules,
            organism_rules,
            specimen_rules)
        
    return converter

In [4]:
session = aiohttp.ClientSession(connector=CONNECTOR)
loop = asyncio.get_event_loop()
converter = await loop.create_task(get_converter(session))
biosample_record, etag = await loop.create_task(get_biosample_record('SAMEA7044752', session))

In [5]:
record = converter.convert_record(biosample_record, etag)

In [6]:
biosample_record.keys()

dict_keys(['name', 'accession', 'domain', 'release', 'update', 'taxId', 'characteristics', 'relationships', 'externalReferences', 'releaseDate', 'updateDate', 'submittedVia', 'create', 'certificates', '_links'])

Try to force record update:

In [7]:
result = await loop.create_task(put_record(session, biosample_record['accession'], record, record['material']))
print(result)

None


In [8]:
session.close()

<coroutine object ClientSession.close at 0x7fd8e8498c20>